# Extensions/limitations of linear regression

```{admonition} Read then Launch 
This content is best viewed in html because jupyter notebook cannot display some content (e.g. figures, equations) properly. You should finish reading this page first and then launch it as an interactive notebook in Google Colab (faster, Google account needed) or Binder by clicking the rocket symbol (<i class="fas fa-rocket"></i>) at the top.
```

This section studies how to extend linear regression to more complicated types of variables and relationships, and the limitations of linear regression. We will use three datasets to illustrate these concepts:

1. The [Advertising dataset](https://github.com/pykale/transparentML/blob/main/data/Advertising.csv) that has been used in the previous sections. 
2. The [Credit dataset](https://github.com/pykale/transparentML/blob/main/data/Credit.csv) contains information about credit card holders. The goal is to predict the credit `Limit` for each card holder. 
3. The [Auto dataset](https://github.com/pykale/transparentML/blob/main/data/Auto.csv) dataset contains information about the characteristics of cars. The goal is to predict the `mpg` (miles per gallon) for a car based on its features like `horsepower`, `weight` and `acceleration`. This dataset has also been used in the tutorial of {doc}`Prerequisites <../00-prereq/overview>`.

You are encouraged to click the links to the datasets and explore the variables and understand their relationships.

## Import libraries and load data

Get ready by importing the APIs needed from respective libraries.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import scale
from sklearn.linear_model import LinearRegression

from statsmodels.formula.api import ols

%matplotlib inline

Load the [Advertising dataset](https://github.com/pykale/transparentML/blob/main/data/Advertising.csv) dataset that we are already familiar with.

In [ ]:
data_url = "https://github.com/pykale/transparentML/raw/main/data/Advertising.csv"
advertising_df = pd.read_csv(data_url, header=0, index_col=0)

Load the [Credit dataset](https://github.com/pykale/transparentML/blob/main/data/Credit.csv) dataset, convert the `Student2` category to numbers ('0' and '1'), and inspect the first three rows.

In [ ]:
credit_url = "https://github.com/pykale/transparentML/raw/main/data/Credit.csv"

credit_df = pd.read_csv(credit_url)
credit_df["Student2"] = credit_df.Student.map({"No": 0, "Yes": 1})
credit_df.head(3)

Load the [Auto dataset](https://github.com/pykale/transparentML/blob/main/data/Auto.csv) datasets, ignore rows with missing values, and inspect the columns, counts and data types.

In [ ]:
auto_url = "https://github.com/pykale/transparentML/raw/main/data/Auto.csv"

auto_df = pd.read_csv(auto_url, na_values="?").dropna()
auto_df.info()

## Qualitative predictors

In the previous sections, the predictor variables are *quantitative*. Here, in the `Credit` dataset, the response is `Balance` (average credit card debt for each individual) and there are six quantitative predictors: `Age` , `Cards` (number of credit cards), `Education` (years of education), `Income` (in thousands of dollars), `Limit` (credit limit), and `Rating` (credit rating). The following code displays the pairwise relationships between these variables (Figure 3.6 in the textbook).

In [ ]:
sns.pairplot(
    credit_df[["Balance", "Age", "Cards", "Education", "Income", "Limit", "Rating"]]
)
plt.show()

However, in practice, predictor variables are not always quantitative. For example, in the `Credit` dataset, it also contains four _qualitative_ (categorical) variables: `Student` (`Yes` or `No`), `Own` (`Yes` or `No`), `Married` (`Yes` or `No`), and `Region` (`South`, `East`, or `West`).

**Predictors with only two levels**

If a qualitative predictor has only two levels (possible values), then we can treat it as a quantitative variable, known as an indicator or _dummy_ variable. For example, the `Student` variable has two levels: `Yes` and `No`. We can convert these levels to numbers, such as `1` and `0`, and then treat the `Student` variable as a quantitative variable. This is called *coding* the qualitative variable, which is automatically done by the `statsmodels` library. 

Run the following code to get the least squares coefficient estimates associated with the regression of `Balance` onto `Own` in the `Credit` dataset (Table 3.7 in the textbook).

In [ ]:
est = ols("Balance ~ Own", credit_df).fit()
est.summary().tables[1]

The large $p$ value above shows that there is no relationship between `Balance` and `Own`. 

**Predictors with more than two levels**

When a qualitative predictor has more than two levels, we need to create additional dummy variables. For example, the `Region` variable has three levels: `South`, `East`, and `West`. We can create **two** dummy variables, `South`, and `West`, and then use these variables in the regression model. There will be always _one fewer dummy variable than the number of levels_ in the qualitative variable. This is because the regression model can determine the value of the missing dummy variable. For example, if `South` is `0` and `West` is `0`, then the individual must be from the `East` region. Again, such _coding_ is automatically done by the `statsmodels` library.

Run the following code to get the least squares coefficient estimates associated with the regression of `Balance` onto `Region` in the `Credit` dataset (Table 3.8 in the textbook).

In [ ]:
est = ols("Balance ~ Region", credit_df).fit()
est.summary().tables[1]

The large $p$-values above show that there is no relationship between `Balance` and `Region`. 

## Extensions of linear regression

### Interaction between variables

In the previous analysis of `Advertising` dataset, the predictor variables are assumed to be independent. However, this model may be incorrect. For example, `radio` advertising can increase the effectiveness of `TV` advertising, which is known as *interaction* effect in statistics. Consider the standard multiple linear regression with two variables

$$
y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \epsilon.
$$

This model can be extended to include an interaction term between $x_1$ and $x_2$:

\begin{align}
y &= \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_1 x_2 + \epsilon \\
&= \beta_0 + (\beta_1 + \beta_3 x_2) x_1 + \beta_2 x_2 + \epsilon \\
&= \beta_0 + \tilde{\beta}_1 x_1 + \beta_2 x_2 + \epsilon,
\end{align}

where $\tilde{\beta}_1 = \beta_1 + \beta_3 x_2$ is the *effective* coefficient on $x_1$, which can be viewed as _another linear regression_ that regresses $\tilde{\beta}_1$ onto $x_2$. Then the association between $y$ and $x_1$ is no longer constant and depends on the value of $x_2$. Similarly, the association between $y$ and $x_2$ is no longer constant and depends on the value of $x_1$.

Run the following code for the `Advertising` data to get the least squares coefficient estimates associated with the regression of `Sales` onto `TV` and `Radio`, with an interaction term, `TV` $\times$ `Radio` (Table 3.9 of the text book).

In [ ]:
est = ols("Sales ~ TV + Radio + TV*Radio", advertising_df).fit()
est.summary().tables[1]

The small $p$-value for the interaction term suggests that the true relationship is not additive and there is strong evidence of the association between `Sales` and the interaction between `TV` and `Radio`. Without including the interaction term, our understanding of the relationship between `Sales` and `TV` and `Radio` is not complete.

#### Interaction between qualitative and quantitative variables

The concept of interactions also applies to qualitative variables. In fact, sometimes an interaction between a qualitative variable and a quantitative variable has a particularly nice interpretation. For example, run the following code to predict `Balance` using the `Income` (quantitative) and `Student` (qualitative) variables, and compare the differences between including and excluding the interaction term between `Income` and `Student`.

In [ ]:
est1 = ols("Balance ~ Income + Student2", credit_df).fit()
est2 = ols("Balance ~ Income + Income*Student2", credit_df).fit()

print("Regression 1 - without interaction term")
print(est1.params)
print("\nRegression 2 - with interaction term")
print(est2.params)

We can see that the interaction term between `Income` and `Student` affects the relationship between `Balance` and `Student` more than the relationship between `Balance` and `Income`. To have a closer look, let us visualise the relationship between `Balance` and `Income` for students and non-students separately (Figure 3.7 of the textbook).

In [ ]:
# Income (x-axis)
regr1 = est1.params
regr2 = est2.params

income = np.linspace(0, 150)

# Balance without interaction term (y-axis)
student1 = np.linspace(
    regr1["Intercept"] + regr1["Student2"],
    regr1["Intercept"] + regr1["Student2"] + 150 * regr1["Income"],
)
non_student1 = np.linspace(
    regr1["Intercept"], regr1["Intercept"] + 150 * regr1["Income"]
)

# Balance with interaction term (y-axis)
student2 = np.linspace(
    regr2["Intercept"] + regr2["Student2"],
    regr2["Intercept"]
    + regr2["Student2"]
    + 150 * (regr2["Income"] + regr2["Income:Student2"]),
)
non_student2 = np.linspace(
    regr2["Intercept"], regr2["Intercept"] + 150 * regr2["Income"]
)

# Create plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(income, student1, "r-", income, non_student1, "k-")
ax2.plot(income, student2, "r--", income, non_student2, "k--")

for ax in fig.axes:
    ax.legend(["student", "non-student"], loc=2)
    ax.set_xlabel("Income")
    ax.set_ylabel("Balance")
    ax.set_ylim(ymax=1550)

In the left figure without the interaction term between `Income` and `Student`, we see two (solid) parallel lines, one for students and one for non-students. The two lines have different intercepts but the same slope, indicating that a one-unit increase in `Income` will have the same effect on `Balance` for students and non-students, which is counterintuitive. 

In the right figure with the interaction term between `Income` and `Student`, the two (dashed) lines are no longer parallel. We see that the slope of the line for students is flatter than that for non-students, indicating that a one-unit increase in `Income` will have a much smaller effect on credit card `Balance` for students than for non-students. This is consistent with our intuition.

### Nonlinear relationships

In some cases, the true relationship between the response and the predictors may be nonlinear. Here we present a very simple way to directly extend the linear model to accommodate nonlinear relationships, using [polynomial regression](https://en.wikipedia.org/wiki/Polynomial_regression).

Run the following code to observe the relationship between `mpg` (gas mileage in miles per gallon) and `horsepower`.

In [ ]:
plt.scatter(
    auto_df.horsepower, auto_df.mpg, facecolors="None", edgecolors="k", alpha=0.5
)
plt.xlabel("horsepower")
plt.ylabel("mpg")

The figure shows a strong relationship between `mpg` and `horsepower`, but it is clearly not linear. We can use a polynomial regression to fit a nonlinear model to the data. We can fit a polynomial regression of degree 2 (quadratic) to the data using the following regression:

\begin{equation}
\textrm{mpg} = \beta_0 + \beta_1 \times \textrm{horsepower} + \beta_2 \times \textrm{horsepower}^2 + \epsilon.
\end{equation}

Run the following code to display the relationship between `mpg` (gas mileage in miles per gallon) and $\textrm{horsepower}^2$ as well as $\textrm{horsepower}^5$ (Figure 3.8 of the textbook) for cars in the `Auto` dataset.

In [ ]:
# With Seaborn's regplot() you can easily plot higher order polynomials.
plt.scatter(
    auto_df.horsepower, auto_df.mpg, facecolors="None", edgecolors="k", alpha=0.5
)

sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Linear",
    scatter=False,
    color="orange",
)
sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Degree 2",
    order=2,
    scatter=False,
    color="lightblue",
)
sns.regplot(
    x=auto_df.horsepower,
    y=auto_df.mpg,
    ci=None,
    label="Degree 5",
    order=5,
    scatter=False,
    color="g",
)
plt.legend()
plt.ylim(5, 55)
plt.xlim(40, 240);

The above figure seems to suggest that a quadratic relationship between `mpg` and `horsepower` is more appropriate than a linear relationship.

In fact, this is still a linear model with $x_1 = \textrm{horsepower}$ and $x_2 = \textrm{horsepower}^2$. Run the following code to create a new predictor `horsepower2` and inspect the [Auto dataset](https://github.com/pykale/transparentML/blob/main/data/Auto.csv). 

In [ ]:
auto_df["horsepower2"] = auto_df.horsepower**2
auto_df.head(3)

Then run the following code to fit a linear regression model with `mpg` as the response and `horsepower` and `horsepower2` as the predictors (Table 3.10 of the textbook).

In [ ]:
est = ols("mpg ~ horsepower + horsepower2", auto_df).fit()
est.summary().tables[1]

### Limitations of linear regression

Every model will have some limitations, and linear regression is no exception. Here we briefly study two of the limitations of linear regression to show the possible complexities of real-world data. More limitations can be found in the textbook.

#### Nonlinearity of the Data
  
The linear regression model assumes that there is a "straight-line" relationship between the predictors and the response. If the true relationship is far from linear, then virtually all of the conclusions that we draw from the fit are questionable. In addition, the prediction accuracy of the model can be significantly reduced.

Run the following code to compare the differences in accuracy between a linear regression model and a quadratic regression model for regressing `mpg` onto `horsepower`.

Firstly, fitting a linear regression model and a quadratic regression model for regressing `mpg` onto `horsepower` and compute the residuals for each model.

In [ ]:
regr = LinearRegression()

# Linear fit
X = auto_df.horsepower.values.reshape(-1, 1)
y = auto_df.mpg
regr.fit(X, y)

auto_df["pred1"] = regr.predict(X)
auto_df["resid1"] = auto_df.mpg - auto_df.pred1

# Quadratic fit
X2 = auto_df[["horsepower", "horsepower2"]].values
regr.fit(X2, y)

auto_df["pred2"] = regr.predict(X2)
auto_df["resid2"] = auto_df.mpg - auto_df.pred2

Create plots of the residuals for each model. What do you observe?

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
sns.regplot(
    x=auto_df.pred1,
    y=auto_df.resid1,
    lowess=True,
    ax=ax1,
    line_kws={"color": "r", "lw": 1},
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax1.hlines(
    0,
    xmin=ax1.xaxis.get_data_interval()[0],
    xmax=ax1.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax1.set_title("Residual Plot for Linear Fit")

# Right plot
sns.regplot(
    x=auto_df.pred2,
    y=auto_df.resid2,
    lowess=True,
    line_kws={"color": "r", "lw": 1},
    ax=ax2,
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax2.hlines(
    0,
    xmin=ax2.xaxis.get_data_interval()[0],
    xmax=ax2.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax2.set_title("Residual Plot for Quadratic Fit")

for ax in fig.axes:
    ax.set_xlabel("Fitted values")
    ax.set_ylabel("Residuals")

```{Admonition} How to interpret the plots?
:class: tip, dropdown
- The left panel displays a residual plot from the linear regression of `mpg` onto `horsepower` on the `Auto` dataset. The red line is a smooth fit to the residuals for identifying any trends in the residuals. Here, the residuals exhibit a clear U-shape, which provides a strong indication of nonlinearity in the data.
- The right panel displays the residual plot that results from the model contains a quadratic term. There appears to be little pattern in the residuals, suggesting that the quadratic term improves the fit to the data.
```

### Collinearity
  
Collinearity refers to the situation in which two or more predictor variables are closely related to one another. 
  
 Run the following code to illustrate the problem of collinearity in the `Credit` dataset (Figure 3.14 of the textbook). 

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
ax1.scatter(credit_df.Limit, credit_df.Age, facecolor="None", edgecolor="r")
ax1.set_ylabel("Age")

# Right plot
ax2.scatter(credit_df.Limit, credit_df.Rating, facecolor="None", edgecolor="r")
ax2.set_ylabel("Rating")

for ax in fig.axes:
    ax.set_xlabel("Limit")
    ax.set_xticks([2000, 4000, 6000, 8000, 12000])

In the left panel, the two predictors `limit` and `age` appear to have no obvious relationship. In contrast, in the right panel, the predictors `limit` and `rating` are very highly correlated with each other, and we say that they are *collinear*. In this context, `limit` and `rating` tend to increase or decrease together, it can be difficult to determine how each one separately is associated with the response `balance`.

Run the following code to learn the difficulties that can result from collinearity in the context of the `Credit` dataset.

In [ ]:
y = credit_df.Balance

# Regression for left plot
X = credit_df[["Age", "Limit"]].values
regr1 = LinearRegression()
regr1.fit(scale(X.astype("float"), with_std=False), y)
print("Age/Limit\n", regr1.intercept_)
print(regr1.coef_)

# Regression for right plot
X2 = credit_df[["Rating", "Limit"]].values
regr2 = LinearRegression()
regr2.fit(scale(X2.astype("float"), with_std=False), y)
print("\nRating/Limit\n", regr2.intercept_)
print(regr2.coef_)

Create grid coordinates for plotting and then calculate RSS based on grid of coefficients

In [ ]:
# grid coordinates
beta_age = np.linspace(regr1.coef_[0] - 3, regr1.coef_[0] + 3, 100)
beta_limit = np.linspace(regr1.coef_[1] - 0.02, regr1.coef_[1] + 0.02, 100)

beta_rating = np.linspace(regr2.coef_[0] - 3, regr2.coef_[0] + 3, 100)
beta_limit2 = np.linspace(regr2.coef_[1] - 0.2, regr2.coef_[1] + 0.2, 100)

X1, Y1 = np.meshgrid(beta_limit, beta_age, indexing="xy")
X2, Y2 = np.meshgrid(beta_limit2, beta_rating, indexing="xy")
Z1 = np.zeros((beta_age.size, beta_limit.size))
Z2 = np.zeros((beta_rating.size, beta_limit2.size))

limit_scaled = scale(credit_df.Limit.astype("float"), with_std=False)
age_scaled = scale(credit_df.Age.astype("float"), with_std=False)
rating_scaled = scale(credit_df.Rating.astype("float"), with_std=False)

# calculate RSS
for (i, j), v in np.ndenumerate(Z1):
    Z1[i, j] = (
        (y - (regr1.intercept_ + X1[i, j] * limit_scaled + Y1[i, j] * age_scaled)) ** 2
    ).sum() / 1000000

for (i, j), v in np.ndenumerate(Z2):
    Z2[i, j] = (
        (y - (regr2.intercept_ + X2[i, j] * limit_scaled + Y2[i, j] * rating_scaled))
        ** 2
    ).sum() / 1000000

Plot the contours of the RSS with respect to the coefficients of the two linear regression models in 2D.

In [ ]:
from matplotlib.pyplot import contour


fig = plt.figure(figsize=(12, 5))
fig.suptitle("RSS - Regression coefficients", fontsize=20)

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

min_RSS = r"$\beta_0$, $\beta_1$ for minimized RSS"

# Left plot
contour1 = ax1.contour(X1, Y1, Z1, cmap=plt.cm.Set1, levels=[21.25, 21.5, 21.8])
ax1.scatter(regr1.coef_[1], regr1.coef_[0], c="r", label=min_RSS)
ax1.clabel(contour1, inline=True, fontsize=10, fmt="%1.1f")
ax1.set_ylabel(r"$\beta_{Age}$", fontsize=17)

# Right plot
contour2 = ax2.contour(X2, Y2, Z2, cmap=plt.cm.Set1, levels=[21.5, 21.8])
ax2.scatter(regr2.coef_[1], regr2.coef_[0], c="r", label=min_RSS)
ax2.clabel(contour2, inline=True, fontsize=10, fmt="%1.1f")
ax2.set_ylabel(r"$\beta_{Rating}$", fontsize=17)
ax2.set_xticks([-0.1, 0, 0.1, 0.2])

for ax in fig.axes:
    ax.set_xlabel(r"$\beta_{Limit}$", fontsize=17)
    ax.legend()

Left: A contour plot of RSS for the regression of `balance` onto `age` and `limit`. The minimum value is well defined. 

Right: A contour plot of RSS for the regression of `balance` onto `rating` and `limit`. Because of the collinearity, there are many pairs ($\beta_{\text{Limit}}$ , $\beta_{\text{Rating}}$) with a similar value for RSS.

Collinearity reduces the accuracy of the estimates of the regression coefficients, it causes the standard error for $\hat{\beta}_j$ to grow. In the presence of collinearity, we may fail to reject $H_0$ : $\hat{\beta}_j = 0$. This means that the power of the hypothesis test—the probability of correctly detecting a non-zero coefficient is reduced by collinearity.

Run the following code to compare the coefficient estimates obtained from two separate multiple regression models using `statsmodels`, where the first model is a regression of `balance` on `age` and `limit`, and the second model is a regression of `balance` on `rating` and `limit`.

In [ ]:
est_age_limit = ols("Balance ~ Age + Limit", credit_df).fit()
est_age_limit.summary().tables[1]

In [ ]:
est_rating_limit = ols("Balance ~ Rating + Limit", credit_df).fit()
est_rating_limit.summary().tables[1]

The above generates Table 3.11 of the textbook.

```{admonition} How to interpret the results? 
:class: tip, dropdown
In the first regression, both `age` and `limit` are highly significant with very small $p$-values. In the second, the collinearity between `limit` and `rating` has caused the standard error for the `limit` coefficient estimate to increase by a factor of 12 and the $p$-value to increase to 0.701. In other words, the importance of the `limit` variable has been masked due to the presence of collinearity.
```

## Exercise

**1**. All the following exercises involve the use of the **[Boston](https://github.com/pykale/transparentML/blob/main/data/Boston.csv)** dataset. We will now try to predict per capita crime rate using the other variables in this dataset. In other words, per capita crime rate is the response, and the other variables are the predictors.

For **each predictor**, fit a **simple linear regression** model to predict the response. Don't forget to handle any **NULL values**. **Hint**: See section [2.2.2](https://pykale.github.io/transparentML/02-linear-reg/multi-linear-regression.html#import-libraries-and-load-data).

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
from statsmodels.formula.api import ols
import pandas as pd

boston_url = "https://github.com/pykale/transparentML/raw/main/data/Boston.csv"

boston_df = pd.read_csv(boston_url).dropna()


models_a = [
    ols(formula="crim ~ {}".format(f), data=boston_df).fit()
    for f in boston_df.columns[1:]
]

for model in models_a:
    print(model.summary().tables[1])

**2**. In which of the models is there a **statistically significant** association between the predictor and the response? Create some plots to back up your assertions.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
print("p < 0.05\n")

for model in models_a:
    if model.pvalues[1] < 0.05:
        print(model.params[1:].index[0])

print("\np > 0.05\n")

for model in models_a:
    if model.pvalues[1] > 0.05:
        print(model.params[1:].index[0])

#  zn, indu,s nox, rm, ag, dis, rad, tax, ptratio, black, lstat, medv  are higly significant with very small p values.


def plot_grid(df, response, cols):
    variables = df.columns.drop(response)
    for i in range(0, len(variables), cols):
        g = sns.pairplot(
            df, y_vars=[response], x_vars=variables[i : i + cols], height=4
        )
        g.map(sns.regplot)
    return


plot_grid(boston_df, "crim", 5)

**3**. Fit a **multiple regression** model to predict the response using all of the predictors. **Hint**: See section [2.3.3](https://pykale.github.io/transparentML/02-linear-reg/extension-limitation.html#extensions-of-linear-regression).

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
response = "crim"
predictors = boston_df.columns.drop(response)
f = "{} ~ {}".format(response, "+".join(predictors))

model_b = ols(formula=f, data=boston_df).fit()
model_b.summary()

**4**. Which features seem to be **statistically significant** in this model?

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
print("p < 0.05\n")
model_b.pvalues[model_b.pvalues < 0.05]

# Only the resultent features seem to be statistically significant in this model.

**5**. How do your results from **Exercise 1** compare to your results from **Exercise 3**? Create a plot displaying the univariate regression coefficients from **Exercise 1** on the $x$-axis, and the multiple regression coefficients from **Exercise 3** on the $y$-axis. That is, each predictor is displayed as a single point in the plot. Its coefficient in a simple linear regression model is shown on the $x$-axis, and its coefficient estimate in the multiple linear regression model is shown on the $y$-axis.

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
# Get coefficients
univariate_params = pd.concat([m.params[1:] for m in models_a])
multivariate_params = model_b.params[1:]
df = pd.DataFrame(
    {
        "Univariate_coef": univariate_params,
        "Multivariate_coef": multivariate_params,
    }
)
print(df)

plt.figure(figsize=(20, 10))
ax = sns.scatterplot(x="Univariate_coef", y="Multivariate_coef", data=df)

# Multivariate regression found 5 of 13 predictors to be significnat where univariate regression found 12 of 13 significant. Multivariate regression seems to find significanlty less predictors to be significant.

**6**. Is there evidence of **nonlinear association** between any of the predictors and the response? To answer this question, for each predictor $X$, fit a model of the form

$$
y = \beta_0 + \beta_1 X + \beta_2 X^2 \beta_3 X^3 + \epsilon 
$$

**Hint**: See section [2.3.3.2](https://pykale.github.io/transparentML/02-linear-reg/extension-limitation.html#nonlinear-relationships).

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
models_d = [
    ols(
        formula="crim ~ {0} + np.power({0}, 2) + np.power({0}, 3)".format(f),
        data=boston_df,
    ).fit()
    for f in boston_df.columns[1:]
]

for model in models_d:
    print(model.summary().tables[1])


print("\nFeatures with p < 0.05\n")

sig = pd.concat([model.pvalues[model.pvalues < 0.05] for model in models_d])

print(pd.DataFrame({"P>|t|": sig.drop("Intercept")}))

# There is evidence of a nonlinear association between the repsonse and INDUS, NOX, AGE, DIS, PTRATIO, medv
# There is evidence of a linear association between the response and ZN

**7**. Compare the differences between a linear regression model and a quadratic regression model for regressing **ptratio** onto **crim**.  Does the quadratic term improves the fit to the data? **Hint**: See section [2.3.3.3](https://pykale.github.io/transparentML/02-linear-reg/extension-limitation.html#limitations-of-linear-regression).

In [ ]:
# Write your code below to answer the question

*Compare your answer with the reference solution below*

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns

regr = LinearRegression()

# Linear fit
X = boston_df.ptratio.values.reshape(-1, 1)
y = boston_df.crim
regr.fit(X, y)

boston_df["pred1"] = regr.predict(X)
boston_df["resid1"] = boston_df.crim - boston_df.pred1


boston_df["ptratio2"] = boston_df.ptratio**2
# Quadratic fit
X2 = boston_df[["ptratio", "ptratio2"]].values
regr.fit(X2, y)

boston_df["pred2"] = regr.predict(X2)
boston_df["resid2"] = boston_df.crim - boston_df.pred2


fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Left plot
sns.regplot(
    x=boston_df.pred1,
    y=boston_df.resid1,
    lowess=True,
    ax=ax1,
    line_kws={"color": "r", "lw": 1},
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax1.hlines(
    0,
    xmin=ax1.xaxis.get_data_interval()[0],
    xmax=ax1.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax1.set_title("Residual Plot for Linear Fit")

# Right plot
sns.regplot(
    x=boston_df.pred2,
    y=boston_df.resid2,
    lowess=True,
    line_kws={"color": "r", "lw": 1},
    ax=ax2,
    scatter_kws={"facecolors": "None", "edgecolors": "k", "alpha": 0.5},
)
ax2.hlines(
    0,
    xmin=ax2.xaxis.get_data_interval()[0],
    xmax=ax2.xaxis.get_data_interval()[1],
    linestyles="dotted",
)
ax2.set_title("Residual Plot for Quadratic Fit")

for ax in fig.axes:
    ax.set_xlabel("Fitted values")
    ax.set_ylabel("Residuals")

# The left plot displays a residual plot from the linear regression of ptratio onto crim on the Boston dataset.
# The red line is a smooth fit to the residuals for identifying any trends in the residuals.
# Here, the residuals exhibit a tendency of U-shape, which provides a strong indication of nonlinearity in the data.


# The right panel displays the residual plot that results from the model contains a quadratic term.
# There appears to be little pattern in the residuals, suggesting that the quadratic term improves the fit to the data.